# cfDNA fragmentation data analysis


## Data Preparation

In [ ]:
import numpy as np
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
print('Numpy version is ' + str(np.__version__))

import pandas as pd
print('Pandas version is ' + str(pd.__version__))

import matplotlib
from matplotlib import pyplot as plt
print('Matplotlib version is ' + str(matplotlib.__version__))

import seaborn as sns
print('Seaborn version is ' + str(sns.__version__))


import scipy
from scipy.stats import kstest
print('Scipy version is ' + str(scipy.__version__))

from Frag_Ex_Imm_Func import *


### Generating List of Sample IDs

In [ ]:
df=pd.read_csv("../Data Preprocessing/mtDNA_modeling_data.csv")
display(df)
id_list = df.id.unique()
id_order = [3,4,5,6,7,8,9,10,11,12,0,1,2]
id_list=[id_list[i] for i in id_order]
print(f'Sample id: {id_list}')

### Storing probability distribution

In [ ]:
prob_list = []
for id in id_list:
    df_id = df[df['id']==id]
    df_id_total = df_id['freq'].to_numpy().sum()
    df_id['prob']=df_id['freq']/df_id_total
    prob_list = prob_list + list(df_id['prob'].to_numpy())
df['prob']=prob_list
df['prob x10e6'] = df['prob']*10**6

### Splitting Dataset into mtDNA and gDNA dataset

First we split our dataset into two parts, gDNA dataset and mtDNA dataset.

For the mtDNA dataset, since all fragment reads are small, we group fragment together by length groups.
We then divide the aggregrated count by the group length size to give the count per base pair.


In [ ]:
df_chrM = df[df['chr']=='chrM']
len_grp_size = 1
chrM_bins=np.linspace(0,16500,int(16500/len_grp_size)+1)
df_chrM['length_group']=np.digitize(df_chrM['length'].to_numpy(),chrM_bins, right=True)*len_grp_size
df_chrM = df_chrM.groupby(["id","chr","length_group","study","condition","detail","specimen"], as_index = False)[["freq","prob"]].sum()
df_chrM["count per b.p."]=df_chrM["freq"]/len_grp_size
df_chrM=df_chrM.sort_values(by=['id','chr','length_group'])
#df_agg["log length frequency"]= np.log(df_agg["freq"].to_numpy())
#df_agg["log length"]= np.log(df_agg["length_group"].to_numpy())
#df_agg= df_agg[df_agg['length_group']<=10000]
#df_chrM = df_chrM[df_chrM["freq"]>=2]
print('Mitochondrial DNA Dataset:')
display(df_chrM)


In [ ]:
df_mtDNA = df[df['chr']=='chrM']
len_grp_size = 10
mtDNA_bins=np.linspace(0,16500,int(16500/len_grp_size)+1)
df_mtDNA['length_group']=np.digitize(df_mtDNA['length'].to_numpy(),mtDNA_bins, right=True)*len_grp_size
df_mtDNA = df_mtDNA.groupby(["id","chr","length_group","study","condition","detail","specimen"], as_index = False)[["freq","prob"]].sum()
df_mtDNA["count per b.p."]=df_mtDNA["freq"]/len_grp_size
df_mtDNA=df_mtDNA.sort_values(by=['id','chr','length_group'])
#df_agg["log length frequency"]= np.log(df_agg["freq"].to_numpy())
#df_agg["log length"]= np.log(df_agg["length_group"].to_numpy())
#df_agg= df_agg[df_agg['length_group']<=10000]
#df_chrM = df_chrM[df_chrM["freq"]>=2]
print('Binned Mitochondrial DNA Dataset:')
display(df_mtDNA)


In [ ]:
df_gDNA = df[df['chr']=='gDNA']
#df_gDNA = df_gDNA[df_gDNA['freq']>=2].copy()
df_gDNA_full = df_gDNA.copy()

gDNA_bin = [0, 250] + [int(x) for x in range(420, 16500, 200)]
print(gDNA_bin)
gDNA_width = np.diff(gDNA_bin)
list = gDNA_bin +[16500]

df_gDNA = df_gDNA[df_gDNA['length']<=16500] #only consider fragments with length less than 200 b.p.
df_gDNA['length_group']=[(list[int(x)-1]+list[int(x)])/2 for x in np.digitize(df_gDNA['length'].to_numpy(),gDNA_bin, right=True)]
df_gDNA['length_interval']=[str([list[int(x)-1],list[int(x)]]) for x in np.digitize(df_gDNA['length'].to_numpy(),gDNA_bin, right=True)]
df_gDNA['length_width']=[list[int(x)]-list[int(x)-1] for x in np.digitize(df_gDNA['length'].to_numpy(),gDNA_bin, right=True)]

df_gDNA = df_gDNA.groupby(["id","chr","length_group","length_interval","study","condition","detail","specimen","length_width"], as_index = False)[["freq","prob"]].sum()
df_gDNA["count per b.p."]=df_chrM["freq"]
df_gDNA=df_gDNA.sort_values(by=['id','chr','length_group'])

print('Genomic DNA Dataset:')
display(df_gDNA)
display(df_gDNA_full)

## mtDNA Fragment Profile Curve-fitting analysis

First we identify the samples with more than 200 observed fragments. We will only study these samples.

In [ ]:
mtDNA_id = []
mtDNA_health = []
for id in id_list:
    #id = 'ISPRO.bc05'
    clin_data = df_chrM[df_chrM['id']==id] #chrM
    health = clin_data['condition'].iloc[0]
    x = clin_data['length_group'].to_numpy()
    x= np.append([0],x)
    y = np.cumsum(clin_data['freq'].to_numpy())
    if y[-1] >= 200:
        #print(id, y[-1], clin_data['condition'])
        mtDNA_id += [id]
        mtDNA_health += [health]    
print(f'ID list: {mtDNA_id}')
print(f'Health condition: {mtDNA_health}')

### Fragment Profile Visualisation and Curve Fitting
Here, we visualise the fragment profile. We introduce a cut-off to discard the longest 5% of fragments to reduce noisy data from inaccurate sampling of large fragments.

Furthermore, we linearly interpolate the cumulative count to create a smoother fragment profile.

We can see that all fragment profiles start to decrease around 120-150 b.p.. This suggests that fragments start to exit the system once they are within this region.

We also find the best fit curve for the decreasing part of the fragment profiles by fitting a linear model on the log-log scale.

In [ ]:
import scipy.interpolate
n = len(mtDNA_id)
fig, ax = plt.subplots(3,3,figsize= (12,12), dpi=400)
counter = 0
bf_coeff = np.zeros((n,2))
low_cutoff=np.ones(n)*140
best_fit_stats = []
for id in mtDNA_id:
    clin_data_binned = df_mtDNA[df_mtDNA['id']==id]
    mtDNA_bins = clin_data_binned['length_group'].to_numpy()
    y_count = clin_data_binned['freq'].to_numpy()
    ax[int(counter/3),counter%3].bar(mtDNA_bins, y_count, width=10, alpha=0.8, label='Fragment profile')
##############################Linear interpolation of fragment counts to provide best fit coefficients that account for 0 counts #####################################################    
    clin_data = df_chrM[df_chrM['id']==id] 
    x = clin_data['length_group'].to_numpy()
    x= np.append([0],x)
    y = np.cumsum(clin_data['freq'].to_numpy())
    y = np.append([0],y)
    cum_count = np.stack([x,y], axis=0)
    y_interp = scipy.interpolate.interp1d(cum_count[0,:], cum_count[1,:])
    up_idx = np.where(y > np.percentile(y,95))[0].min()
    frag_cutoff = x[up_idx]
    x_axis = np.linspace(0,frag_cutoff,int(frag_cutoff/10)+1)
    y_int = y_interp(x_axis)
    y_lim = y_int[-1]*1.5
    #ax[1,counter].plot(x_axis,y_int, color = 'k', label='linear interpolation')
    #ax[1,counter].bar(x[:up_idx],y[:up_idx],width=10, alpha=0.5, label='cumulative count')
    #ax[1, counter].legend(loc='upper left',fontsize=6)
    #ax[1, counter].set_ylim([0,y_lim])
    freq_int = np.diff(y_int)
    x_max = x_axis[np.where(freq_int == np.max(freq_int))[0]]
    print(id, x_max)
    peak = low_cutoff[counter]
    x_tail = np.linspace(peak,frag_cutoff,int(frag_cutoff/10)+1)
    y_tail = y_interp(x_tail)  
    log_den_trans = np.log(np.diff(y_tail))-np.log(np.diff(y_tail))[0]  #shifting the density point at B_ex to origin
    log_x_axis_trans = np.log(x_tail[1:]) - np.log(x_tail[1]) #shifting the point to origin
    lm = LinearRegression(fit_intercept = False) #apply a linear model with no intercept fit
    lm.fit(log_x_axis_trans.reshape(-1, 1), log_den_trans)
    [m] = lm.coef_
    c = np.log(np.diff(y_tail))[0] - m * np.log(x_tail[1])
    bf_coeff[counter,:] = [m,c]
    print(m,c)
    #plt.plot(x,y/y[-1], label=f'CDF for {id}')
    #plt.title(f'Cumulative Fragment Count for Patients mtDNA')
    #plt.xlabel('Fragment Length (b.p.)')
    #plt.ylabel('Fragment Count')
    #print(np.linspace(0,8000,801),y_int)
    #plt.plot(x,y, label=f'Id: {id}')
    #ax[0, counter].plot(x,y, label=f'Cumulative count')
    #ax[0,counter].set_title(f'Cumulative count, Sample {id}')
    #ax[counter].set_xscale('log')
    #ax[counter].set_yscale('log')
    ax[int(counter/3),counter%3].set_ylim([0.01,50])
    ax[int(counter/3),counter%3].set_xlim([1,500])
    ax[int(counter/3),counter%3].plot(x_tail, np.exp(c)*x_tail**m, label=f'Best Fit: {np.exp(c): .1e} x^{m: .2f}', color='k', linestyle='-.')
    ax[int(counter/3),counter%3].set_title(f'Sample {id}')
    #ax[counter].set_xlim([100,frag_cutoff])
    ax[int(counter/3),counter%3].axvline(peak, label=f'{peak} b.p.', color='k')
    ax[int(counter/3),counter%3].legend(loc='upper right',fontsize=10)
 
    fig.tight_layout()
    best_fit_stats += [[mtDNA_health[counter], id, m,c]]
    counter +=1
    fig.suptitle('mtDNA Fragment Profile')


#fig.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_frag_prof_nanopore.eps')
#fig.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_frag_prof_nanopore.png')
fig.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_frag_prof_nanopore.pdf')

#fig.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_frag_prof_nanopore_loglog.png')

print(bf_coeff)



In [ ]:
import scipy.interpolate
fig, ax = plt.subplots(3,3,figsize= (12,12), dpi=400)
counter = 0
bf_coeff = np.zeros((n,2))
low_cutoff=np.ones(n)*140
best_fit_stats = []
for id in mtDNA_id:
    clin_data_binned = df_mtDNA[df_mtDNA['id']==id]
    mtDNA_bins = clin_data_binned['length_group'].to_numpy()
    y_count = clin_data_binned['freq'].to_numpy()
    ax[int(counter/3),counter%3].bar(mtDNA_bins, y_count, width=10, alpha=0.8, label='Fragment profile')
##############################Linear interpolation of fragment counts to provide best fit coefficients that account for 0 counts #####################################################    
    clin_data = df_chrM[df_chrM['id']==id] 
    x = clin_data['length_group'].to_numpy()
    x= np.append([0],x)
    y = np.cumsum(clin_data['freq'].to_numpy())
    y = np.append([0],y)
    cum_count = np.stack([x,y], axis=0)
    y_interp = scipy.interpolate.interp1d(cum_count[0,:], cum_count[1,:])
    up_idx = np.where(y > np.percentile(y,95))[0].min()
    frag_cutoff = x[up_idx]
    x_axis = np.linspace(0,frag_cutoff,int(frag_cutoff/10)+1)
    y_int = y_interp(x_axis)
    freq_int = np.diff(y_int)
    x_max = x_axis[np.where(freq_int == np.max(freq_int))[0]]
    print(id, x_max)
    peak = low_cutoff[counter]
    x_tail = np.linspace(peak,frag_cutoff,int(frag_cutoff/10)+1)
    y_tail = y_interp(x_tail)  
    log_den_trans = np.log(np.diff(y_tail))-np.log(np.diff(y_tail))[0]  #shifting the density point at B_ex to origin
    log_x_axis_trans = np.log(x_tail[1:]) - np.log(x_tail[1]) #shifting the point to origin
    lm = LinearRegression(fit_intercept = False) #apply a linear model with no intercept fit
    lm.fit(log_x_axis_trans.reshape(-1, 1), log_den_trans)
    [m] = lm.coef_
    c = np.log(np.diff(y_tail))[0] - m * np.log(x_tail[1])
    bf_coeff[counter,:] = [m,c]
    print(m,c)
    #plt.plot(x,y/y[-1], label=f'CDF for {id}')
    #plt.title(f'Cumulative Fragment Count for Patients mtDNA')
    #plt.xlabel('Fragment Length (b.p.)')
    #plt.ylabel('Fragment Count')
    #print(np.linspace(0,8000,801),y_int)
    #plt.plot(x,y, label=f'Id: {id}')
    #ax[0, counter].plot(x,y, label=f'Cumulative count')
    #ax[0,counter].set_title(f'Cumulative count, Sample {id}')
    ax[int(counter/3),counter%3].set_ylim([0.1,1000])
    ax[int(counter/3),counter%3].set_xlim([50,8000])
    ax[int(counter/3),counter%3].set_xscale('log')
    ax[int(counter/3),counter%3].set_yscale('log')
    ax[int(counter/3),counter%3].plot(x_tail, np.exp(c)*x_tail**m, label=f'Best Fit: {np.exp(c): .1e} x^{m: .2f}', color='k', linestyle='-.')
    ax[int(counter/3),counter%3].set_title(f'Sample {id}')
    #ax[counter].set_xlim([100,frag_cutoff])
    ax[int(counter/3),counter%3].axvline(peak, label=f'{peak} b.p.', color='k')
    ax[int(counter/3),counter%3].legend(loc='upper right',fontsize=10)  #ax[1, counter].legend(fontsize=10)
    fig.tight_layout()
    best_fit_stats += [[mtDNA_health[counter], id, m,c]]
    counter +=1
    fig.suptitle('mtDNA Fragment Profile')



#fig.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_frag_prof_nanopore_1.png')
#fig.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_frag_prof_nanopore_1.pdf')
#fig.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_frag_prof_nanopore_loglog.png')
fig.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_frag_prof_nanopore_loglog.pdf')

print(bf_coeff)



In [ ]:

df_bf_mtDNA = pd.DataFrame(best_fit_stats, columns = ['condition','id','mtDNA slope', 'mtDNA intercept'])
mean = df_bf_mtDNA['mtDNA slope'].mean()
print(f'mtDNA slope mean: {mean: .2f}')
g2 = sns.scatterplot(data=df_bf_mtDNA, x='mtDNA slope', y='mtDNA intercept', style= 'condition')
g2.set(title='Distribution of mtDNA statistics')
g2.legend(fontsize=10)
#plt.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_stats_dist.eps')
#plt.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_stats_dist.png')
plt.savefig('../Clinical Data Analysis Results/Population Plot/mtDNA_stats_dist.pdf')

df_bf_mtDNA.head()



## gDNA Fragment Profile Curve-fitting analysis

### Per Patient Visualisation

In [ ]:
best_fit_stats = []


for id in id_list:
    fig, ax = plt.subplots(2, dpi=150, figsize=[6,6])
    df_id_gDNA = df_gDNA[df_gDNA['id']==id]
    [id_health]=df_id_gDNA.condition.unique()
    print(id_health)
    avg_count = df_id_gDNA['freq'].to_numpy()/(df_id_gDNA['freq'].sum()*df_id_gDNA['length_width'])*1000
    avg_bin = df_id_gDNA['length_group'].to_numpy()
    m_0,c_0 = np.polyfit(np.log(avg_bin[1:]),np.log(avg_count[1:]),1)
    #gDNA_tail = df_id_gDNA[df_id_gDNA['length_group']>=167]
    #gDNA_tail_length = gDNA_tail['length_group'].to_numpy()
    #gDNA_tail_count = gDNA_tail['freq'].to_numpy()/(gDNA_tail['freq'].sum()*gDNA_tail['length_width'])*1000
    #m_0, c_0 = np.polyfit(np.log(gDNA_tail_length),np.log(gDNA_tail['freq'].to_numpy()),1)
    
    clin_data_full = df_gDNA_full[df_gDNA_full['id']==id]
    clin_data_full = clin_data_full[clin_data_full['length']<=10000]
    clin_data_full.sort_values(by='length')
    clin_length_full = clin_data_full['length'].to_numpy()
    clin_freq_full = clin_data_full['freq'].to_numpy()    
    
    ax[0].set_title('cfDNA fragment profile for sample: ' + id, fontsize=10)
    #print(df_id_gDNA['length_group'].to_numpy())
    ax[0].bar(x=avg_bin, height=avg_count,label='gDNA average', alpha=0.7, color='salmon', width = df_id_gDNA['length_width'].to_numpy()*0.95)
    #ax[0].scatter(gDNA_tail_length, np.exp(c_0)*gDNA_tail_length**m_0,label=f'Best Fit: {np.exp(c_0): .1e}x^{m_0: .2f}', color = 'k', s=0.5)
    ax[0].set_xlabel('DNA fragment length (b.p.)', fontsize=6)
    ax[0].set_xlim([0,600])
    ax[0].bar(clin_length_full,clin_freq_full/np.sum(clin_freq_full)*1000, label=f"gDNA count", width=1, alpha=0.5, color = 'b')
    #ax[0].set_yscale('log')
    #ax[0].bar(x=df_id_chrM['length_group'].to_numpy(), height=df_id_chrM['freq'].to_numpy(),label='mtDNA',width=0.8*len_grp_size, alpha=1, color='blue')
    #ax[0].plot(chrM_tail_length, np.exp(c_1)*chrM_tail_length**m_1,label=f'mtDNA count best fit curve: {np.exp(c_1): .1e}x^{m_1: .2f}', color = 'darkblue')
    ax[0].set_ylabel('Count per 1000 Fragments', fontsize=6)
    ax[0].legend(loc='upper right',fontsize=10)

    ax[1].set_title('log-log cfDNA fragment profile for sample: ' + id, fontsize=10)
    ax[1].bar(x=avg_bin, height=avg_count,label='gDNA average', alpha=0.7, color='salmon', width=df_id_gDNA['length_width'].to_numpy()*0.95)
    ax[1].plot(avg_bin[1:], np.exp(c_0)*avg_bin[1:]**m_0, color = 'red')
    ax[1].set_xlabel('DNA fragment length (b.p.)', fontsize=6)
    #ax[1].bar(x=df_id_chrM['length_group'].to_numpy(), height=df_id_chrM['freq'].to_numpy(),label='mtDNA',width=0.8*len_grp_size, alpha=1, color='blue')
    #ax[1].plot(chrM_tail_length, np.exp(c_1)*chrM_tail_length**m_1,label=f'log y = {c_1: .1e}+ {m_1: .2f} log x', color = 'darkblue')
    ax[1].set_ylabel('Count per 1000 Fragments', fontsize=6)
    ax[1].legend(loc='upper right',fontsize=10)
    ax[1].set_yscale('log')
    ax[1].set_xscale('log')
    ax[1].set_ylim([10**-6,10**2])
    best_fit_stats += [[id_health, id, m_0,c_0]]
    #ax1[id_list.index(id)].legend(fontsize=10)
    #fig1.suptitle('Mitochondrial cfDNA fragment profile for healthy patients', fontsize =30)
    fig.tight_layout()
    path = r'../Clinical Data Analysis Results/Individual Plots'
    fig_name = str(id) +'_ind_plt'
    #fig.savefig(path + '/'+ fig_name +'.eps', transparent=False)
    #fig.savefig(path + '/'+ fig_name +'.png', transparent=False)  
    fig.savefig(path + '/'+ fig_name +'.pdf', transparent=True)  





### All Samples in One Picture

In [ ]:
fig, ax = plt.subplots(5,3, dpi=200, figsize=[30,50])
counter=0
fig.suptitle('\n Genomic cfDNA fragment profile', fontsize =40)

for id in id_list:
    df_id_gDNA = df_gDNA[df_gDNA['id']==id]
    [id_health]=df_id_gDNA.condition.unique()
    print(id_health)
    avg_count = df_id_gDNA['freq'].to_numpy()/(df_id_gDNA['freq'].sum()*df_id_gDNA['length_width'])*1000
    avg_bin = df_id_gDNA['length_group'].to_numpy()
    m_0,c_0 = np.polyfit(np.log(avg_bin[1:]),np.log(avg_count[1:]),1)
    #gDNA_tail = df_id_gDNA[df_id_gDNA['length_group']>=167]
    #gDNA_tail_length = gDNA_tail['length_group'].to_numpy()
    #gDNA_tail_count = gDNA_tail['freq'].to_numpy()/(gDNA_tail['freq'].sum()*gDNA_tail['length_width'])*1000
    #m_0, c_0 = np.polyfit(np.log(gDNA_tail_length),np.log(gDNA_tail['freq'].to_numpy()),1)
    
    clin_data_full = df_gDNA_full[df_gDNA_full['id']==id]
    clin_data_full = clin_data_full[clin_data_full['length']<=10000]
    clin_data_full.sort_values(by='length')
    clin_length_full = clin_data_full['length'].to_numpy()
    clin_freq_full = clin_data_full['freq'].to_numpy()    
    
    ax[int(counter/3),counter%3].set_title(id, fontsize=25)
    ax[int(counter/3),counter%3].bar(x=avg_bin, height=avg_count,label='gDNA average', alpha=0.7, color='salmon', width=df_id_gDNA['length_width'].to_numpy()*0.95)
    ax[int(counter/3),counter%3].plot(avg_bin[1:], np.exp(c_0)*avg_bin[1:]**m_0, color = 'k')
    ax[int(counter/3),counter%3].set_xlabel('DNA fragment length (b.p.)', fontsize=20)
    #ax[1].bar(x=df_id_chrM['length_group'].to_numpy(), height=df_id_chrM['freq'].to_numpy(),label='mtDNA',width=0.8*len_grp_size, alpha=1, color='blue')
    #ax[1].plot(chrM_tail_length, np.exp(c_1)*chrM_tail_length**m_1,label=f'log y = {c_1: .1e}+ {m_1: .2f} log x', color = 'darkblue')
    ax[int(counter/3),counter%3].set_ylabel('Count per 1000 Fragments', fontsize=20)
    ax[int(counter/3),counter%3].legend(loc='upper right', fontsize=20)
    ax[int(counter/3),counter%3].set_yscale('log')
    ax[int(counter/3),counter%3].set_ylim([10**-7,10**2])
    counter+=1
    #ax1[id_list.index(id)].legend(fontsize=10)
    #fig1.suptitle('Mitochondrial cfDNA fragment profile for healthy patients', fontsize =30)

#fig.tight_layout()
fig.subplots_adjust(top=0.925)
path = r'../Clinical Data Analysis Results/Population Plot'
#fig.savefig(path + '/gDNA_frag_prof_nanopore.png', transparent=False)
#fig.savefig(path + '/gDNA_frag_prof_nanopore.eps', transparent=False) 
fig.savefig(path + '/gDNA_frag_prof_nanopore.pdf', transparent=False)



### Scatter Plots of Fragment Profile Statistics

In [ ]:
df_bf_gDNA = pd.DataFrame(best_fit_stats, columns = ['condition','id','gDNA slope', 'gDNA intercept'])
df_bf_gDNA.to_csv('../Clinical Data Analysis Results/Parameters CSV/gDNA_best_fit_coeff.csv')

In [ ]:
g1 = sns.scatterplot(data=df_bf_gDNA, x='gDNA slope', y='gDNA intercept', hue='condition')
mean = df_bf_gDNA['gDNA slope'].mean()
print(f'gDNA slope mean: {mean: .2f}')
g1.set(title='Distribution of gDNA statistics')
g1.legend(loc='upper right',bbox_to_anchor=(1, 0.8))
#plt.savefig(path + '/gDNA_stats_dist_nanopore.png')
#plt.savefig(path + '/gDNA_stats_dist_nanopore.eps')
plt.savefig(path + '/gDNA_stats_dist_nanopore.pdf')



### mtDNA gDNA comparison

In [ ]:
df_bf_gDNA['DNA']='gDNA'
display(df_bf_gDNA)
df_bf_gDNA = df_bf_gDNA.rename(columns={"gDNA slope": "slope", "gDNA intercept": "intercept"})
display(df_bf_gDNA)
df_bf_mtDNA['DNA']='mtDNA'
df_bf_mtDNA = df_bf_mtDNA.rename(columns={"mtDNA slope": "slope", "mtDNA intercept": "intercept"})
df_bf=pd.concat([df_bf_gDNA,df_bf_mtDNA])
display(df_bf)

In [ ]:

g2 = sns.scatterplot(data=df_bf, x='slope', y='intercept', style='condition', hue='DNA')
g2.set(title='gDNA slope vs mtDNA slope')
g2.legend(loc='upper right',bbox_to_anchor=(1, 0.8))
#plt.savefig(path + '/mtDNA_gDNA_slope.png')
#plt.savefig(path + '/mtDNA_gDNA_slope.eps')
plt.savefig(path + '/mtDNA_gDNA_slope.pdf')



## mtDNA FRIME Best Fit Simulation

We now try to use our FRIME process simulation to simulate the fragment profiles.

We only work on 4 fragment profiles with more than 500 fragment counts for accurate statistical inference.





In [ ]:
## Printing lists of id with 500+ mtDNA fragments

mtDNA_id_500 = []
for id in id_list:
    #id = 'ISPRO.bc05'
    clin_data = df_chrM[df_chrM['id']==id] #chrM
    health = clin_data['condition'].iloc[0]
    x = clin_data['length_group'].to_numpy()
    x= np.append([0],x)
    y = np.cumsum(clin_data['freq'].to_numpy())
    if y[-1] >= 500:
        #print(id, y[-1], clin_data['condition'])
        mtDNA_id_500 += [id]
print(f'ID list: {mtDNA_id_500}')


In [ ]:
#Only use the following two lines to re-create a new sim_clin_df file
sim_clin_df = pd.DataFrame(columns = ['time','id','clin_list','sim_list','seed','N_event','KS_dist','p-value','ex_max','ex_slope','frag_speed','N_split','a','b','alpha_f','Nimm','imm_rate','lin_ex','B_ex','alpha_e','x_max'])
sim_clin_df.to_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin.csv', index=False)


sim_clin_df = pd.read_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin.csv')
display(sim_clin_df)


### Best Fit for Sample 'ISPRO.bc05'

In [ ]:
from Frag_Ex_Imm_Func import *
from itertools import groupby
id = 'ISPRO.bc05'
clin_data = df_chrM[df_chrM['id']==id] #chrM
clin_data = clin_data[clin_data['length_group']<=10000]
#clin_data = df_gDNA[df_gDNA['id']==id] #gDNA
display(clin_data)
clin_data = clin_data.sort_values('length_group') #For chrM Data
#clin_data = clin_data.sort_values('length') #For gDNA Data
clin_data['cumsum']=clin_data['freq'].cumsum()
clin_cum = clin_data[['length_group','cumsum']].to_numpy() #chrM
#clin_cum = clin_data[['length','cumsum']].to_numpy() #gDNA
clin_cum = np.append([[0,0]],clin_cum, axis=0)


############## Writing the cumulative count function from frequency table ##############################
#x_max = clin_data['length'].max() #gDNA
x_max = clin_data['length_group'].max() #mtDNA
clin_ccf = np.zeros(x_max+1)
for i in range(np.shape(clin_cum)[0]):
    clin_ccf[clin_cum[i,0]]=clin_cum[i,1]

for j in range(x_max+1):
    clin_ccf[j]=clin_ccf[0:j+1].max()

################ Generating list of fragment lengths (with repetition) from frequency table #########################
clin_list = []
#chrM
for length in clin_data['length_group']:
    clin_data_length = clin_data[clin_data['length_group']==length]
    clin_list += [length]*clin_data_length['freq'].to_numpy()[0]
#gDNA
#for length in clin_data['length']:
    #clin_data_length = clin_data[clin_data['length']==length]
    #clin_list += [length]*clin_data_length['freq'].to_numpy()[0]

In [ ]:
from datetime import datetime
from scipy import stats
time = int(datetime.timestamp(datetime.now()))
############################# Simulating Best Fit ################################
ex_max = 140
ex_slope = 16500
#B_ex = ex_max/ex_slope
N_split = 2
a=1
b=1
alpha_e = -2.10
xlim_max = 10000
B_ex= np.inf
alpha_f = -0.78
Nimm = 1
clin_max = clin_data['length_group'].max() #chrM
#x_max = clin_data['length_group'].max() #chrM
#x_max = clin_data['length'].max() #gDNA
#x_max=15507
x_max = 10000
frag_speed = 750000
imm_rate = 8
KS_threshold = 0.01
Time_scale = 0.1
N_Group = x_max
Max_event = 10**6
N_event = 0
t=0
lin_ex=False
KS_list = [0.1]
time_list = []
p_list = []
plot=False
seed=42
np.random.seed(seed)
Prior_Count = 1
x = Frag_List(np.random.uniform(0,x_max, Prior_Count),np.zeros(Prior_Count))
clin_max = clin_data['length_group'].max() #chrM
#clin_max = clin_data['length'].max() #gDNA
Event_interval = 10000
sim_mat = np.zeros((int(Max_event/Event_interval)+1,x_max+1))
while N_event < Max_event:
    Event, Event_time = x.FragExIm(ex_max, ex_slope, frag_speed, N_split, a,b, alpha_f, Nimm, imm_rate,t,lin_ex, B_ex, alpha_e, x_max = x_max)
    N_event +=1
    t += Event_time
    if N_event % Event_interval ==0:
        sim_cdf = x.cum_dist(x_max, x_max)
        time_list += [t]
        sim_mat[int(N_event/Event_interval),:]=sim_cdf
        if np.min(x.Frag_labs) != 0:
            KS_list += [KS_Dist(sim_cdf[:clin_max+1],clin_ccf/clin_ccf[-1])]
            sim_list = np.round(x.Frag_lens)
            p_value = stats.ks_2samp(clin_list,sim_list)[1]
            p_list += [p_value]
            print(f'# Event: {N_event}, Time: {t:.3f}, KS_dist: {KS_list[-1]:.3f}, p_value: {p_value}')
            print('Stationarity Reached')
            print(f'Oldest fragment immigrated at time {np.min(x.Frag_labs): .3e}')
            print(f'KS_mean: {np.mean(KS_list[-20:])}')
        if plot == True:
            plt.clf()
            plt.plot(sim_cdf, label='sim')
            plt.plot(clin_ccf[0:x_max+1]/clin_ccf[x_max+1], label='real')
            plt.legend(fontsize=10)
            plt.show()
    

In [ ]:
sim_list = np.round(x.Frag_lens)
p_value = stats.ks_2samp(clin_list,sim_list)[1]

list = [time, id, clin_list, sim_list, seed, N_event, KS_list[-1],p_value, ex_max,ex_slope,frag_speed,N_split,a,b,alpha_f,Nimm,imm_rate,lin_ex,B_ex,alpha_e,x_max]
sim_clin_df.loc[len(sim_clin_df)]=list
sim_clin_df.to_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin.csv', index=False)
print(p_value)


In [ ]:
sim_list = np.round(x.Frag_lens)
from scipy import stats
p_value = stats.ks_2samp(clin_list,sim_list)[1]
up_bound = np.percentile(sim_mat, 75, axis=0)
low_bound = np.percentile(sim_mat, 25, axis=0)
mean = np.mean(sim_mat, axis=0)
sim_cdf = x.cum_dist(x_max, x_max)
fig, ax = plt.subplots(2,figsize=(8,12), dpi=100)
#############Plotting New Density Against Mean Density over Time ############################
ax[0].plot(range(clin_max+1),clin_ccf/clin_ccf[-1], label=f"Clinical")
ax[0].plot(range(x_max+1),sim_cdf, label = f"Simulated", color='r')
ax[0].plot(range(clin_max+1),sim_cdf[:clin_max+1]-clin_ccf/clin_ccf[-1], label='Difference')
ax[0].set_xlim(0,10000)
ax[0].set_xlabel('Fragment Length')
ax[0].set_ylabel('Cumulative Frequency')
ax[0].set_title('Cumulative Profile')
ax[0].plot(range(x_max+1),up_bound, linestyle='-.', alpha=0.5, color = '0.8',label='90-th %')
ax[0].plot(range(x_max+1),low_bound, linestyle='-.', alpha=0.5, color = '0.8',label='10-th %')
ax[0].fill_between(range(x_max+1), low_bound, up_bound, alpha=0.2, color = '0.5')      
ax[0].legend(fontsize=10)
ax[0].axhline(0, color='k')
ax[1].set_xlim(0,10000)
ax[1].bar(range(x_max),np.diff(sim_cdf)*1000, label = f"Simulated",color='r', width=2)
ax[1].bar(range(clin_max)[:clin_max+1],np.diff(clin_ccf/clin_ccf[-1])*1000, label=f"Clinical", width=2, alpha=0.5)
#ax[1].plot(range(x_max),np.diff(mean)*1000, linestyle='-.', alpha=0.5, color = '0.8',label='Mean')
#ax[0].set_xlim(0,x_max)
ax[1].set_xlabel('Fragment Length')
ax[1].set_ylabel('Count per 1000 Fragments')
ax[1].set_title(f'Fragment Profile')
ax[1].legend(fontsize=10)
#fig.suptitle(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.bc05. \n alpha= {alpha_f: .2f}, E(x) = x^{alpha_e:.1f}-{B_ex}^{alpha_e:.1f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, \n Seed={seed}, N_Event = {N_event}, p-value ={p_value: .3f}')
fig.suptitle(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.bc05. \n alpha= {alpha_f: .2f}, E(x) = x^{alpha_e:.1f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, \n Seed={seed}, N_Event = {N_event}, p-value ={p_value: .3f}')
fig.tight_layout()
fig.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRObc05.pdf')

In [ ]:
plt.plot(np.linspace(6*10**4,10*10**4,95),p_list, label='p_value')
plt.axhline(0.05, label='0.05', color='r')
plt.xlabel('Number of Event')
plt.ylabel('p value')
plt.title('p_value evolution for mtDNA fragment profile ISPRO.bc05')
plt.legend(fontsize=10)
plt.savefig('../Clinical Data Analysis Results/p_value evolution/p_value_evol_mtDNA_ISPRO_bc05.pdf')

### Best Fit Simulation for ISPRO.S1

In [ ]:
from Frag_Ex_Imm_Func import *
from itertools import groupby
id = 'ISPRO.S1'
clin_data = df_chrM[df_chrM['id']==id] #chrM
clin_data = clin_data[clin_data['length_group']<=10000]
#clin_data = df_gDNA[df_gDNA['id']==id] #gDNA
display(clin_data)
clin_data = clin_data.sort_values('length_group') #For chrM Data
#clin_data = clin_data.sort_values('length') #For gDNA Data
clin_data['cumsum']=clin_data['freq'].cumsum()
clin_cum = clin_data[['length_group','cumsum']].to_numpy() #chrM
#clin_cum = clin_data[['length','cumsum']].to_numpy() #gDNA
clin_cum = np.append([[0,0]],clin_cum, axis=0)


############## Writing the cumulative count function from frequency table ##############################
#x_max = clin_data['length'].max() #gDNA
x_max = clin_data['length_group'].max() #mtDNA
clin_ccf = np.zeros(x_max+1)
for i in range(np.shape(clin_cum)[0]):
    clin_ccf[clin_cum[i,0]]=clin_cum[i,1]

for j in range(x_max+1):
    clin_ccf[j]=clin_ccf[0:j+1].max()

################ Generating list of fragment lengths (with repetition) from frequency table #########################
clin_list = []
#chrM
for length in clin_data['length_group']:
    clin_data_length = clin_data[clin_data['length_group']==length]
    clin_list += [length]*clin_data_length['freq'].to_numpy()[0]
#gDNA
#for length in clin_data['length']:
    #clin_data_length = clin_data[clin_data['length']==length]
    #clin_list += [length]*clin_data_length['freq'].to_numpy()[0]

In [ ]:
from datetime import datetime

time = int(datetime.timestamp(datetime.now()))
############################# Simulating Best Fit ################################
ex_max = 375
ex_slope = 16500
#B_ex = ex_max/ex_slope
N_split = 2
a=1
b=1
alpha_e = -2.05
xlim_max = 10000
B_ex= np.inf
alpha_f = -1.00
Nimm = 1
#x_max = clin_data['length_group'].max() #chrM
#x_max = clin_data['length'].max() #gDNA
#x_max=15507
x_max = 10000
frag_speed = 1000000
imm_rate = 11
KS_threshold = 0.01
Time_scale = 0.1
N_Group = x_max
Max_event = 10**6
N_event = 0
t=0
lin_ex=False
KS_list = []
time_list = []
plot=False
seed=42
np.random.seed(seed)
Prior_Count = 1
x = Frag_List(np.random.uniform(0,x_max, Prior_Count),np.zeros(Prior_Count))
clin_max = clin_data['length_group'].max() #chrM
#clin_max = clin_data['length'].max() #gDNA
Event_interval = 5*10000
sim_mat = np.zeros((int(Max_event/Event_interval)+1,x_max+1))
while N_event < Max_event:
    Event, Event_time = x.FragExIm(ex_max, ex_slope, frag_speed, N_split, a,b, alpha_f, Nimm, imm_rate,t,lin_ex, B_ex, alpha_e, x_max = x_max)
    N_event +=1
    t += Event_time
    if N_event % Event_interval ==0:
        sim_cdf = x.cum_dist(x_max, x_max)
        time_list += [t]
        sim_mat[int(N_event/Event_interval),:]=sim_cdf
        if np.min(x.Frag_labs) != 0:
            KS_list += [KS_Dist(sim_cdf[:clin_max+1],clin_ccf/clin_ccf[-1])]
            print(f'# Event: {N_event}, Time: {t:.3f}, KS_dist: {KS_list[-1]:.3f}')
            print('Stationarity Reached')
            print(f'Oldest fragment immigrated at time {np.min(x.Frag_labs): .3e}')
            print(f'KS_mean: {np.mean(KS_list[-10:])}')
        if plot == True:
            plt.clf()
            plt.plot(sim_cdf, label='sim')
            plt.plot(clin_ccf[0:x_max+1]/clin_ccf[x_max+1], label='real')
            plt.legend(fontsize=10)
            plt.show()
    

In [ ]:
plt.plot(KS_list)
plt.axhline(0.03)

In [ ]:
sim_list = np.round(x.Frag_lens)
#mean_list = np.round(Frag_list)
from scipy import stats
from scipy import stats
p_value = stats.ks_2samp(clin_list,sim_list)[1]
#p_value_mean = stats.ks_2samp(clin_list,mean_list)[1]
print(p_value)
list = [time, id, clin_list, sim_list, seed, N_event, KS_list[-1],p_value, ex_max,ex_slope,frag_speed,N_split,a,b,alpha_f,Nimm,imm_rate,lin_ex,B_ex,alpha_e,x_max]
sim_clin_df.loc[len(sim_clin_df)]=list
sim_clin_df.to_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin.csv', index=False)
#print(p_value, p_value_mean)


In [ ]:
sim_list = np.round(x.Frag_lens)
#mean_list = np.round(Frag_list)
from scipy import stats
p_value = stats.ks_2samp(clin_list,sim_list)[1]
print(p_value)
x_max = 9989
up_bound = np.percentile(sim_mat, 75, axis=0)
low_bound = np.percentile(sim_mat, 25, axis=0)
mean = np.mean(sim_mat, axis=0)
sim_cdf = x.cum_dist(x_max, x_max)[:9990]
fig, ax = plt.subplots(2,figsize=(8,12), dpi=100)
#############Plotting New Density Against Mean Density over Time ############################
ax[0].plot(range(clin_max+1),clin_ccf/clin_ccf[-1], label=f"Clinical")
ax[0].plot(range(x_max+1),sim_cdf, label = f"Simulated", color='r')
ax[0].plot(range(x_max+1),sim_cdf-clin_ccf/clin_ccf[-1], label='Difference')
ax[0].set_xlim(0,10000)
ax[0].set_xlabel('Fragment Length')
ax[0].set_ylabel('Cumulative Frequency')
ax[0].set_title('Cumulative Profile')
ax[0].plot(range(x_max+1),up_bound[:9990], linestyle='-.', alpha=0.5, color = '0.8',label='75-th %')
ax[0].plot(range(x_max+1),low_bound[:9990], linestyle='-.', alpha=0.5, color = '0.8',label='25-th %')
ax[0].fill_between(range(x_max+1), low_bound[:9990], up_bound[:9990], alpha=0.2, color = '0.5')      
ax[0].legend(fontsize=10)
ax[0].axhline(0, color='k')
ax[1].set_xlim(0,10000)
ax[1].bar(range(x_max),np.diff(sim_cdf)*1000, label = f"Simulated",color='r', width=2)
ax[1].bar(range(clin_max),np.diff(clin_ccf/clin_ccf[-1])*1000, label=f"Clinical", width=2, alpha=0.5)
#ax[1].plot(range(x_max),np.diff(mean)*1000, linestyle='-.', alpha=0.5, color = '0.8',label='Mean')
#ax[0].set_xlim(0,x_max)
ax[1].set_xlabel('Fragment Length')
ax[1].set_ylabel('Count per 1000 Fragments')
ax[1].set_title(f'Fragment Profile')
ax[1].legend(fontsize=10)
#fig.suptitle(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.S1. \n alpha= {alpha_f: .3f}, E(x) = x^{alpha_e:.1f}-{B_ex}^{alpha_e:.1f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, r~ Beta({a},{b}) \n Seed={seed}, N_Event = {N_event}, p-value ={p_value: .3f}')
fig.suptitle(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.S1. \n alpha= {alpha_f: .3f}, E(x) = x^{alpha_e:.2f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, r~ Beta({a},{b}) \n Seed={seed}, N_Event = {N_event}, p-value ={p_value: .3f}')

fig.tight_layout()
fig.savefig('../Clinical Data Analysis Results/Individual Plots/ISPROS1.pdf')

### Best Fit Simulation for Sample ISPRO.bc03

In [ ]:
from Frag_Ex_Imm_Func import *
from itertools import groupby
id = 'ISPRO.bc03'
clin_data = df_chrM[df_chrM['id']==id] #chrM
clin_data = clin_data[clin_data['length_group']<=10000]
#clin_data = df_gDNA[df_gDNA['id']==id] #gDNA
display(clin_data)
clin_data = clin_data.sort_values('length_group') #For chrM Data
#clin_data = clin_data.sort_values('length') #For gDNA Data
clin_data['cumsum']=clin_data['freq'].cumsum()
clin_cum = clin_data[['length_group','cumsum']].to_numpy() #chrM
#clin_cum = clin_data[['length','cumsum']].to_numpy() #gDNA
clin_cum = np.append([[0,0]],clin_cum, axis=0)


############## Writing the cumulative count function from frequency table ##############################
#x_max = clin_data['length'].max() 
x_max = clin_data['length_group'].max() 

clin_ccf = np.zeros(x_max+1)
for i in range(np.shape(clin_cum)[0]):
    clin_ccf[clin_cum[i,0]]=clin_cum[i,1]

for j in range(x_max+1):
    clin_ccf[j]=clin_ccf[0:j+1].max()

################ Generating list of fragment lengths (with repetition) from frequency table #########################
clin_list = []
#chrM
for length in clin_data['length_group']:
    clin_data_length = clin_data[clin_data['length_group']==length]
    clin_list += [length]*clin_data_length['freq'].to_numpy()[0]
#gDNA
#for length in clin_data['length']:
    #clin_data_length = clin_data[clin_data['length']==length]
    #clin_list += [length]*clin_data_length['freq'].to_numpy()[0]

In [ ]:
from datetime import datetime

time = int(datetime.timestamp(datetime.now()))
############################# Simulating Best Fit ################################
ex_max = 135
ex_slope = 16500
#B_ex = ex_max/ex_slope
N_split = 2
a=1
b=1
alpha_e = -2.1
xlim_max = 10000
B_ex= np.inf
alpha_f = -0.50
Nimm = 1
clin_max = clin_data['length_group'].max() #chrM
x_max=10000
#x_max = clin_data['length'].max() #gDNA
#x_max=15507
#x_max = 8000
frag_speed = 100000
imm_rate = 10
KS_threshold = 0.01
Time_scale = 0.1
N_Group = x_max
Max_event = 10**6
N_event = 0
t=0
lin_ex=False
KS_list = []
time_list = []
plot=False
seed=42
np.random.seed(seed)
Prior_Count = 1
x = Frag_List(np.random.uniform(0,x_max, Prior_Count),np.zeros(Prior_Count))
clin_max = clin_data['length_group'].max() #chrM
#clin_max = clin_data['length'].max() #gDNA
Event_interval = 5*10000
sim_mat = np.zeros((int(Max_event/Event_interval)+1,x_max+1))
while N_event < Max_event:
    Event, Event_time = x.FragExIm(ex_max, ex_slope, frag_speed, N_split, a,b, alpha_f, Nimm, imm_rate,t,lin_ex, B_ex, alpha_e, x_max = x_max)
    N_event +=1
    t += Event_time
    if N_event % Event_interval ==0:
        sim_cdf = x.cum_dist(x_max, x_max)
        time_list += [t]
        sim_mat[int(N_event/Event_interval),:]=sim_cdf
        if np.min(x.Frag_labs) != 0:
            KS_list += [KS_Dist(sim_cdf[:clin_max+1],clin_ccf/clin_ccf[-1])]
            print(f'# Event: {N_event}, Time: {t:.3f}, KS_dist: {KS_list[-1]:.3f}')
            print('Stationarity Reached')
            print(f'Oldest fragment immigrated at time {np.min(x.Frag_labs): .3e}')
            print(f'KS_mean: {np.mean(KS_list)}')
        if plot == True:
            plt.clf()
            plt.plot(sim_cdf, label='sim')
            plt.plot(clin_ccf[0:x_max+1]/clin_ccf[x_max+1], label='real')
            plt.legend(fontsize=10)
            plt.show()
    

In [ ]:
sim_list = np.round(x.Frag_lens)
#mean_list = np.round(Frag_list)
from scipy import stats
p_value = stats.ks_2samp(clin_list,sim_list)[1]
#p_value_mean = stats.ks_2samp(clin_list,mean_list)[1]
print(p_value)
list = [time, id, clin_list, sim_list, seed, N_event, KS_list[-1],p_value, ex_max,ex_slope,frag_speed,N_split,a,b,alpha_f,Nimm,imm_rate,lin_ex,B_ex,alpha_e,x_max]
sim_clin_df.loc[len(sim_clin_df)]=list
sim_clin_df.to_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin.csv', index=False)
#print(p_value, p_value_mean)


In [ ]:
print(sim_cdf.shape)
print(x_max)

In [ ]:
up_bound = np.percentile(sim_mat, 90, axis=0)
low_bound = np.percentile(sim_mat, 10, axis=0)
mean = np.mean(sim_mat, axis=0)
sim_cdf = x.cum_dist(x_max, x_max)
fig, ax = plt.subplots(2,figsize=(8,12), dpi=100)
#############Plotting New Density Against Mean Density over Time ############################
ax[0].plot(range(clin_max+1),clin_ccf/clin_ccf[-1], label=f"Clinical")
ax[0].plot(range(x_max+1),sim_cdf, label = f"Simulated", color='r')
ax[0].plot(range(clin_max+1),sim_cdf[:clin_max+1]-clin_ccf/clin_ccf[-1], label='Difference')
ax[0].set_xlim(10,10000)
ax[0].set_xlabel('Fragment Length')
ax[0].set_ylabel('Cumulative Frequency')
ax[0].set_title('Cumulative Profile')
ax[0].plot(range(x_max+1),up_bound, linestyle='-.', alpha=0.5, color = '0.8',label='90-th %')
ax[0].plot(range(x_max+1),low_bound, linestyle='-.', alpha=0.5, color = '0.8',label='10-th %')
ax[0].fill_between(range(x_max+1), low_bound, up_bound, alpha=0.2, color = '0.5')      
ax[0].legend(loc='upper left',fontsize=10)
ax[0].axhline(0, color='k')
ax[1].set_xlim(10,10000)
ax[1].bar(range(x_max),np.diff(sim_cdf)*1000, label = f"Simulated",color='r', width=2)
ax[1].bar(range(clin_max),np.diff(clin_ccf/clin_ccf[-1])*1000, label=f"Clinical", width=2, alpha=0.5)
#ax[1].plot(range(x_max),np.diff(mean)*1000, linestyle='-.', alpha=0.5, color = '0.8',label='Mean')
#ax[0].set_xlim(0,x_max)
ax[1].set_xlabel('Fragment Length')
ax[1].set_ylabel('Count per 1000 Fragments')
ax[1].set_title(f'Fragment Profile')
ax[1].legend(fontsize=10)
fig.suptitle(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.bc03. \n alpha= {alpha_f: .2f}, E(x) = x^{alpha_e:.1f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, r~ Beta({a},{b}) \n Seed={seed}, N_Event = {N_event}, p-value ={p_value: .3f}')
fig.tight_layout()
fig.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRObc03.pdf')

### Best Fit Simulation for Sample ISPRO.bc01

In [ ]:
from Frag_Ex_Imm_Func import *
from itertools import groupby
id = 'ISPRO.bc01'
clin_data = df_chrM[df_chrM['id']==id] #chrM
clin_data = clin_data[clin_data['length_group']<=10000]
#clin_data = df_gDNA[df_gDNA['id']==id] #gDNA
display(clin_data)
clin_data = clin_data.sort_values('length_group') #For chrM Data
#clin_data = clin_data.sort_values('length') #For gDNA Data
clin_data['cumsum']=clin_data['freq'].cumsum()
clin_cum = clin_data[['length_group','cumsum']].to_numpy() #chrM
#clin_cum = clin_data[['length','cumsum']].to_numpy() #gDNA
clin_cum = np.append([[0,0]],clin_cum, axis=0)


############## Writing the cumulative count function from frequency table ##############################
#x_max = clin_data['length'].max() #gDNA
x_max = clin_data['length_group'].max() #mtDNA
clin_ccf = np.zeros(x_max+1)
for i in range(np.shape(clin_cum)[0]):
    clin_ccf[clin_cum[i,0]]=clin_cum[i,1]

for j in range(x_max+1):
    clin_ccf[j]=clin_ccf[0:j+1].max()

################ Generating list of fragment lengths (with repetition) from frequency table #########################
clin_list = []
#chrM
for length in clin_data['length_group']:
    clin_data_length = clin_data[clin_data['length_group']==length]
    clin_list += [length]*clin_data_length['freq'].to_numpy()[0]
#gDNA
#for length in clin_data['length']:
    #clin_data_length = clin_data[clin_data['length']==length]
    #clin_list += [length]*clin_data_length['freq'].to_numpy()[0]

In [ ]:
from datetime import datetime

time = int(datetime.timestamp(datetime.now()))
############################# Simulating Best Fit ################################
ex_max = 110
ex_slope = 16500
#B_ex = ex_max/ex_slope
N_split = 2
a=1
b=1
alpha_e = -2.5
xlim_max = 4000
B_ex= 115
alpha_f = 0.36
Nimm = 1
clin_max = clin_data['length_group'].max() #chrM
x_max = 10000
#x_max = clin_data['length'].max() #gDNA
#x_max=15507
#x_max = 8000
frag_speed = 1000
imm_rate = 10
KS_threshold = 0.01
Time_scale = 0.1
N_Group = x_max
Max_event = 10**6
N_event = 0
t=0
lin_ex=False
KS_list = [0.1]
time_list = []
plot=False
seed=42
np.random.seed(seed)
Prior_Count = 1
x = Frag_List(np.random.uniform(0,x_max, Prior_Count),np.zeros(Prior_Count))
clin_max = clin_data['length_group'].max() #chrM
#clin_max = clin_data['length'].max() #gDNA
Event_interval = 10000
sim_mat = np.zeros((int(Max_event/Event_interval)+1,x_max+1))
while N_event < Max_event:
    Event, Event_time = x.FragExIm(ex_max, ex_slope, frag_speed, N_split, a,b, alpha_f, Nimm, imm_rate,t,lin_ex, B_ex, alpha_e, x_max = x_max)
    N_event +=1
    t += Event_time
    if N_event % Event_interval ==0:
        sim_cdf = x.cum_dist(x_max, x_max)
        time_list += [t]
        sim_mat[int(N_event/Event_interval),:]=sim_cdf
        if np.min(x.Frag_labs) != 0:
            KS_list += [KS_Dist(sim_cdf[:clin_max+1],clin_ccf/clin_ccf[-1])]
            print(f'# Event: {N_event}, Time: {t:.3f}, KS_dist: {KS_list[-1]:.3f}')
            print('Stationarity Reached')
            print(f'Oldest fragment immigrated at time {np.min(x.Frag_labs): .3e}')
            print(f'KS_mean: {np.mean(KS_list)}')
        if plot == True:
            plt.clf()
            plt.plot(sim_cdf, label='sim')
            plt.plot(clin_ccf[0:x_max+1]/clin_ccf[x_max+1], label='real')
            plt.legend(fontsize=10)
            plt.show()
    

In [ ]:
sim_list = np.round(x.Frag_lens)
#mean_list = np.round(Frag_list)
from scipy import stats
p_value = stats.ks_2samp(clin_list,sim_list)[1]
#p_value_mean = stats.ks_2samp(clin_list,mean_list)[1]
print(p_value)
list = [time, id, clin_list, sim_list, seed, N_event, KS_list[-1],p_value, ex_max,ex_slope,frag_speed,N_split,a,b,alpha_f,Nimm,imm_rate,lin_ex,B_ex,alpha_e,x_max]
sim_clin_df.loc[len(sim_clin_df)]=list
sim_clin_df.to_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin.csv', index=False)
#print(p_value, p_value_mean)


In [ ]:
up_bound = np.percentile(sim_mat, 75, axis=0)
low_bound = np.percentile(sim_mat, 25, axis=0)
mean = np.mean(sim_mat, axis=0)
sim_cdf = x.cum_dist(x_max, x_max)
fig, ax = plt.subplots(2,figsize=(8,12), dpi=100)
#############Plotting New Density Against Mean Density over Time ############################
ax[0].plot(range(clin_max+1),clin_ccf/clin_ccf[-1], label=f"Clinical")
ax[0].plot(range(x_max+1),sim_cdf, label = f"Simulated", color='r')
ax[0].plot(range(clin_max+1),sim_cdf[:clin_max+1]-clin_ccf/clin_ccf[-1], label='Difference')
ax[0].set_xlim(0,4000)
ax[0].set_ylim(0,1.2)
ax[0].set_xlabel('Fragment Length')
ax[0].set_ylabel('Cumulative Frequency')
ax[0].set_title('Cumulative Profile')
ax[0].plot(range(x_max+1),up_bound, linestyle='-.', alpha=0.5, color = '0.8',label='75-th %')
ax[0].plot(range(x_max+1),low_bound, linestyle='-.', alpha=0.5, color = '0.8',label='25-th %')
ax[0].fill_between(range(x_max+1), low_bound, up_bound, alpha=0.2, color = '0.5')      
ax[0].legend(loc='upper left',fontsize=10)
ax[0].axhline(0, color='k')
ax[1].set_xlim(0,4000)
ax[1].bar(range(x_max),np.diff(sim_cdf)*1000, label = f"Simulated",color='r', width=2)
ax[1].bar(range(clin_max),np.diff(clin_ccf/clin_ccf[-1])*1000, label=f"Clinical", width=2, alpha=0.5)
#ax[1].plot(range(x_max),np.diff(mean)*1000, linestyle='-.', alpha=0.5, color = '0.8',label='Mean')
#ax[0].set_xlim(0,x_max)
ax[1].set_xlabel('Fragment Length')
ax[1].set_ylabel('Count per 1000 Fragments')
ax[1].set_title(f'Fragment Profile')
ax[1].legend(fontsize=10)
fig.suptitle(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.bc01. \n alpha= {alpha_f: .2f}, E(x) = x^{alpha_e:.1f}-{B_ex}^{alpha_e:.1f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, r~ Beta({a},{b}) \n Seed={seed}, N_Event = {N_event}, p-value ={p_value: .3f}')
fig.tight_layout()
fig.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRObc01.pdf')

### mtDNA summary



In [ ]:
sim_fit_df = sim_clin_df[sim_clin_df['p-value']>=0.1]
sim_fit_df.sort_values(by=['p-value'])
sim_fit_df = sim_fit_df[sim_fit_df['x_max']==10000]
sim_fit_df = sim_fit_df[sim_fit_df['B_ex']!='100000000000000000000']
sim_fit_df = sim_fit_df[sim_fit_df['N_event']==10**6]
sim_fit_df = sim_fit_df.drop(['ex_slope','clin_list','sim_list','ex_max','N_split','lin_ex','Nimm','time'],axis=1)
sim_fit_df = sim_fit_df.round(3)
sim_fit_df.to_csv('../Clinical Data Analysis Results/Parameters CSV/sim_fit_mtDNA.csv', index=False)

## gDNA FRIME Best Fit Simulation

We now try to use our FRIME process simulation to simulate the fragment profiles.

We only work on gDNA dataset for ISPRO.bc05。





In [ ]:
#Only use the following two lines to re-create a new sim_clin_df file
sim_clin_gDNA = pd.DataFrame(columns = ['time','id','clin_list','sim_list','seed','N_event','KS_dist','p-value','ex_max','ex_slope','frag_speed','N_split','a','b','alpha_f','Nimm','imm_rate','lin_ex','B_ex','alpha_e','x_max'])
sim_clin_gDNA.to_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin_gDNA.csv', index=False)


sim_clin_gDNA = pd.read_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin_gDNA.csv')
display(sim_clin_gDNA)


In [ ]:
from Frag_Ex_Imm_Func import *
from itertools import groupby
id = 'ISPRO.bc05'
clin_data_full = df_gDNA_full[df_gDNA_full['id']==id]
clin_data_full = clin_data_full[clin_data_full['length']<=10000]
clin_data_full.sort_values(by='length')
display(clin_data_full)
clin_length_full = clin_data_full['length'].to_numpy()
clin_freq_full = clin_data_full['freq'].to_numpy()

clin_data = df_gDNA[df_gDNA['id']==id] #gDNA
clin_data = clin_data[clin_data['length_group']<=10000]
display(clin_data)
clin_data = clin_data.sort_values('length_group')
clin_data['cumsum']=clin_data['freq'].cumsum()
clin_cum = clin_data[['length_group','cumsum']].to_numpy() 
clin_cum = np.append([[0,0]],clin_cum, axis=0)


############## Writing the cumulative count function from frequency table ##############################
#x_max = clin_data['length'].max() #gDNA
x_max = clin_data['length_group'].max() #mtDNA
clin_ccf = np.zeros(int(x_max+1))
for i in range(np.shape(clin_cum)[0]):
    clin_ccf[int(clin_cum[i,0])]=clin_cum[i,1]

for j in range(int(x_max+1)):
    clin_ccf[j]=clin_ccf[0:j+1].max()

################ Generating list of fragment lengths (with repetition) from frequency table #########################
clin_list = []
#chrM
for length in clin_data['length_group']:
    clin_data_length = clin_data[clin_data['length_group']==length]
    clin_list += [length]*clin_data_length['freq'].to_numpy()[0]

# gDNA
#for length in clin_data['length']:
    #clin_data_length = clin_data[clin_data['length']==length]
    #clin_list += [length]*clin_data_length['freq'].to_numpy()[0]

clin_count = clin_data['freq'].to_numpy()
clin_length = clin_data['length_group'].to_numpy()
print(clin_count)

In [ ]:
# Individual Plot

plt.bar(clin_length_full,clin_freq_full/np.sum(clin_freq_full)*1000, label=f"Clinical", width=1, alpha=0.5)
plt.xlim([100,2000])
plt.xscale('log')
plt.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRO_bc05_clean_log.pdf')

In [ ]:
# Individual Plot

plt.bar(clin_length_full,clin_freq_full/np.sum(clin_freq_full)*1000, label=f"Clinical", width=1, alpha=0.5)
plt.xlim([100,600])
plt.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRO_bc05_clean.pdf')

In [ ]:
from datetime import datetime
from scipy import stats
time = int(datetime.timestamp(datetime.now()))
############################# Simulating Best Fit ################################
ex_max = 140
ex_slope = 16500
#B_ex = ex_max/ex_slope
N_split = 2
a=1
b=1
alpha_e = -2
B_ex= 167
alpha_f = 1.20
Nimm = 1
#x_max = int(clin_data['length_group'].max())
#x_max = clin_data['length'].max() #gDNA
x_max = 10000
#x_max=15507
#x_max = 8000
frag_speed = 2.5
imm_rate = 10
KS_threshold = 0.01
Time_scale = 0.1
N_Group = x_max
Max_event = 10**6
N_event = 0
t=0
gDNA_bin = [0, 250] + [int(x) for x in range(420, 10200, 200)]
lin_ex=False
KS_list = [0.02]
chi2_list = [0.02]
p_list = [0]
time_list = []
plot=False
seed=42
np.random.seed(seed)
Prior_Count = 1
x = Frag_List(np.random.uniform(0,x_max, Prior_Count),np.zeros(Prior_Count))
clin_max = clin_data['length_group'].max() #chrM
#clin_max = clin_data['length'].max() #gDNA
Event_interval = 10000
sim_mat = np.zeros((int(Max_event/Event_interval)+1,int(x_max)+1))
while N_event < Max_event:
    Event, Event_time = x.FragExIm(ex_max, ex_slope, frag_speed, N_split, a,b, alpha_f, Nimm, imm_rate,t,lin_ex, B_ex, alpha_e, x_max = x_max)
    N_event +=1
    t += Event_time
    if N_event % Event_interval ==0:
        sim_count, edges = np.histogram(x.Frag_lens,bins=gDNA_bin,density=False)
        sim_list=[(gDNA_bin[int(x)-1]+gDNA_bin[int(x)])/2 for x in np.digitize(x.Frag_lens,gDNA_bin, right=True)]
        chi2_stat = stats.chisquare(f_obs = sim_count[0:3], f_exp=clin_count[0:3]/np.sum(clin_count[0:3])*np.sum(sim_count[0:3]))[0]
        chi2_p = stats.chisquare(f_obs = sim_count[0:3], f_exp=clin_count[0:3]/np.sum(clin_count[0:3])*np.sum(sim_count[0:3]))[1]
        KS_statistics, KS_p = stats.ks_2samp(sim_list,clin_list)
        time_list += [t]
        print(f'# Event: {N_event}, Time: {t:.3e}, KS_dist: {KS_statistics:.3f}, KS p-value: {KS_p:.3e}, chi2_stat = {chi2_stat:.1f}, chi2_p = {chi2_p:.2e}')
        KS_list += [KS_statistics]
        chi2_list += [chi2_stat]
        p_list +=[chi2_p]
        print(f'Oldest fragment immigrated at time {np.min(x.Frag_labs): .3e}')
        print(f'KS_mean: {np.mean(KS_list[-10:])}')

In [ ]:
plt.plot(np.linspace(0,100*10**4,101),p_list, label='p_value')
plt.axhline(0.05, label='0.05', color='r')
plt.xlabel('Number of Event')
plt.ylabel('p value')
plt.title('p_value evolution for ISPRO.bc05 gDNA fragment profile')
plt.legend(fontsize=10)
plt.savefig('../Clinical Data Analysis Results/p_value evolution/p_value_evol_gDNA_ISPRO_bc05.pdf')

In [ ]:
p_value = KS_p

data = [time, id, clin_list, sim_list, seed, N_event, KS_list[-1],p_value, ex_max,ex_slope,frag_speed,N_split,a,b,alpha_f,Nimm,imm_rate,lin_ex,B_ex,alpha_e,x_max]
sim_clin_gDNA.loc[len(sim_clin_gDNA)]=data
sim_clin_gDNA.to_csv('../Clinical Data Analysis Results/Parameters CSV/sim_clin_gDNA.csv', index=False)
print(p_value)


In [ ]:
p_value = KS_p
sim_len_list, sim_count_list = np.unique(sim_list, return_counts=True)
sim_cum_count = sim_count_list.cumsum()
x_max = int(x_max)
clin_max = int(clin_max)
up_bound = np.percentile(sim_mat, 75, axis=0)
low_bound = np.percentile(sim_mat, 25, axis=0)
mean = np.mean(sim_mat, axis=0)
sim_cdf = x.cum_dist(x_max,x_max)
fig, ax = plt.subplots(2,figsize=(8,12), dpi=100)

#############Plotting New Density Against Mean Density over Time ############################
ax[0].bar(clin_length_full,np.cumsum(clin_freq_full)/np.sum(clin_freq_full), label=f"Clinical", width=0.9)
ax[0].plot(np.linspace(0,x_max,x_max+1),sim_cdf/sim_cdf[-1], label = f"Simulated", color='k')
#ax[0].plot(range(x_max+1),sim_cdf-clin_ccf/clin_ccf[-1], label='Difference')
#ax[0].set_xlim(0,8000)
ax[0].set_xlabel('Fragment Length')
ax[0].set_ylabel('Cumulative Frequency')
ax[0].set_title('Cumulative Profile')
ax[0].set_xscale('log')
#ax[0].plot(range(x_max+1),up_bound, linestyle='-.', alpha=0.5, color = '0.8',label='75-th %')
#ax[0].plot(range(x_max+1),low_bound, linestyle='-.', alpha=0.5, color = '0.8',label='25-th %')
#ax[0].fill_between(range(x_max+1), low_bound, up_bound, alpha=0.2, color = '0.5')      
ax[0].legend(fontsize=10)
ax[1].set_xlim(100,5000)
ax[0].set_xlim(100,5000)
ax[1].bar(np.linspace(0,x_max,x_max+1)[1:],np.diff(sim_cdf)/sim_cdf[-1]*1000, label = f"Simulated",color='r', alpha=0.5, width=1)
ax[1].bar(clin_length_full,clin_freq_full/np.sum(clin_freq_full)*1000, label=f"Clinical", width=1, alpha=0.5)
#ax[1].set_yscale('log')
ax[1].set_xscale('log')
#ax[1].bar(np.linspace(0,x_max,100), np.diff(sim_cdf)/sim_cdf[-1]*1000, label=f'Simulated', color='k', width=100)
#ax[1].plot(range(x_max),np.diff(mean)*1000, linestyle='-.', alpha=0.5, color = '0.8',label='Mean')
#ax[0].set_xlim(0,x_max)
ax[1].set_xlabel('Fragment Length')
ax[1].set_ylabel('Count per 1000 Fragments')
#ax[1].set_xscale('log')
#ax[1].set_yscale('log')
plt.legend(fontsize=10)
fig.suptitle(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.bc05. \n alpha= {alpha_f: .4f}, E(x) = x^{alpha_e:.2f}-{B_ex:.2f}^{alpha_e:.2f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, \n Seed={seed}, N_Event = {N_event}, p-value ={chi2_p: .3f}')
fig.tight_layout()
fig.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRObc05_gDNA_full_cum.pdf')

In [ ]:
plt.bar(sim_len_list,sim_count_list/sim_cum_count[-1]*1000, label = f"Simulated",color='r', width=np.diff(gDNA_bin)[:len(sim_len_list)]*0.5)
plt.bar(clin_length+np.diff(gDNA_bin)*0.4,clin_count/np.sum(clin_count)*1000, label=f"Clinical", width=np.diff(gDNA_bin)*0.5, alpha=0.5)
#ax[1].bar(np.linspace(0,x_max,100), np.diff(sim_cdf)/sim_cdf[-1]*1000, label=f'Simulated', color='k', width=100)
#ax[1].plot(range(x_max),np.diff(mean)*1000, linestyle='-.', alpha=0.5, color = '0.8',label='Mean')
#ax[0].set_xlim(0,x_max)
plt.xlabel('Fragment Length')
plt.ylabel('Count per 1000 Fragments')
#ax[1].set_xscale('log')
#ax[1].set_yscale('log')
plt.xlim([0,1500])
plt.title(f'Fragment Profile')
plt.legend(fontsize=10)
plt.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRObc05_sim_clin_bar.pdf')

In [ ]:
p_value = KS_p
sim_len_list, sim_count_list = np.unique(sim_list, return_counts=True)
sim_cum_count = sim_count_list.cumsum()
x_max = int(x_max)
clin_max = int(clin_max)
up_bound = np.percentile(sim_mat, 75, axis=0)
low_bound = np.percentile(sim_mat, 25, axis=0)
mean = np.mean(sim_mat, axis=0)
sim_cdf = x.cum_dist(x_max,x_max)
fig, ax = plt.subplots(2,figsize=(8,12), dpi=100)

#############Plotting New Density Against Mean Density over Time ############################
ax[0].bar(clin_length_full,np.cumsum(clin_freq_full)/np.sum(clin_freq_full), label=f"Clinical", width=0.9)
ax[0].plot(np.linspace(0,x_max,x_max+1),sim_cdf/sim_cdf[-1], label = f"Simulated", color='k')
#ax[0].plot(range(x_max+1),sim_cdf-clin_ccf/clin_ccf[-1], label='Difference')
#ax[0].set_xlim(0,8000)
ax[0].set_xlabel('Fragment Length')
ax[0].set_ylabel('Cumulative Frequency')
ax[0].set_ylim([0,1.2])
ax[0].set_title('Cumulative Profile')
#ax[0].plot(range(x_max+1),up_bound, linestyle='-.', alpha=0.5, color = '0.8',label='75-th %')
#ax[0].plot(range(x_max+1),low_bound, linestyle='-.', alpha=0.5, color = '0.8',label='25-th %')
#ax[0].fill_between(range(x_max+1), low_bound, up_bound, alpha=0.2, color = '0.5')      
ax[0].legend(loc='upper left',fontsize=10)
ax[1].set_xlim(50,5000)
ax[0].set_xlim(50,5000)
ax[1].bar(sim_len_list,sim_count_list/sim_cum_count[-1]*1000, label = f"Simulated",color='r', width=np.diff(gDNA_bin)[:len(sim_len_list)]*0.9)
ax[1].bar(clin_length,clin_count/np.sum(clin_count)*1000, label=f"Clinical", width=np.diff(gDNA_bin)*0.9, alpha=0.5)
#ax[1].bar(np.linspace(0,x_max,100), np.diff(sim_cdf)/sim_cdf[-1]*1000, label=f'Simulated', color='k', width=100)
#ax[1].plot(range(x_max),np.diff(mean)*1000, linestyle='-.', alpha=0.5, color = '0.8',label='Mean')
#ax[0].set_xlim(0,x_max)
ax[1].set_xlabel('Fragment Length')
ax[1].set_ylabel('Count per 1000 Fragments')
#ax[1].set_xscale('log')
#ax[1].set_yscale('log')
ax[1].set_title(f'Fragment Profile')
ax[1].legend(loc='upper right',fontsize=10)
fig.suptitle(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.bc05. \n alpha= {alpha_f: .4f}, E(x) = x^{alpha_e:.2f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, \n Seed={seed}, N_Event = {N_event}, p-value ={p_value: .3f}')
fig.tight_layout()
fig.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRObc05_gDNA_full_cum_v1.pdf')

In [ ]:
p_value = KS_p
sim_len_list, sim_count_list = np.unique(sim_list, return_counts=True)
sim_cum_count = sim_count_list.cumsum()
x_max = int(x_max)
clin_max = int(clin_max)
up_bound = np.percentile(sim_mat, 75, axis=0)
low_bound = np.percentile(sim_mat, 25, axis=0)
mean = np.mean(sim_mat, axis=0)
sim_cdf = x.cum_dist(x_max,x_max)


#############Plotting New Density Against Mean Density over Time ############################
plt.xlim(10,5000)
plt.bar(np.linspace(0,x_max,x_max+1)[1:],np.diff(sim_cdf)/sim_cdf[-1]*1000, label = f"Simulated",color='r', alpha=0.5, width=1)
plt.bar(clin_length_full,clin_freq_full/np.sum(clin_freq_full)*1000, label=f"Clinical", width=1, alpha=0.5)
#ax[1].set_yscale('log')
plt.xscale('log')
#ax[1].bar(np.linspace(0,x_max,100), np.diff(sim_cdf)/sim_cdf[-1]*1000, label=f'Simulated', color='k', width=100)
#ax[1].plot(range(x_max),np.diff(mean)*1000, linestyle='-.', alpha=0.5, color = '0.8',label='Mean')
#ax[0].set_xlim(0,x_max)
plt.xlabel('Fragment Length')
plt.ylabel('Count per 1000 Fragments')
#ax[1].set_xscale('log')
#ax[1].set_yscale('log')
plt.legend(fontsize=10)
plt.title(f'Simulated vs Clinical Fragment Profile, Sample ISPRO.bc05. \n alpha= {alpha_f: .4f}, E(x) = x^{alpha_e:.2f}-{B_ex}^{alpha_e: .2f}, C_F = {frag_speed: .1e}, C_I = {imm_rate}, \n Seed={seed}, N_Event = {N_event}, p-value ={p_value: .3f}')
plt.tight_layout()
plt.savefig('../Clinical Data Analysis Results/Individual Plots/ISPRObc05_gDNA_full_v2.pdf')